In [1]:
import pandas as pd
from IPython.core.display import display, HTML

In [2]:
london_headers =["soa_code", "borough", "major_category", "minor_category", "value", "year", "month"]
baltimore_headers = ["CrimeDate", "CrimeTime", "CrimeCode", "Location", "Description", "Inside/Outside", "Weapon", "Post", "District", "Neighborhood", "Longitude", "Latitude", "Location 1", "Premise" , "vri_name1", "Total Incidents"] 
la_headers = ["DR Number","Date Reported","Date Occurred","Time Occurred","Area ID","Area Name","Reporting District","Crime Code","Crime Code Description","MO Codes","Victim Age","Victim Sex","Victim Descent","Premise Code","Premise Description","Weapon Used Code","Weapon Description","Status Code","Status Description","Crime Code 1","Crime Code 2","Crime Code 3","Crime Code 4","Address","Cross Street","Location" ]
global_headers = ['id', 'type', 'date', 'city', 'district', 'weapon']

class City():
    def __init__(self, name, headers):
        self.name = name
        self.headers = headers

london = City("London", london_headers)
baltimore = City("Baltimore", baltimore_headers)
la = City("LA", la_headers)
glob = City("Global", global_headers)

In [3]:
# https://github.com/luozhouyang/python-string-similarity
from strsimpy.jaro_winkler import JaroWinkler

def getJaroWinkler(s0, s1):
    jaro_winkler = JaroWinkler()
    return jaro_winkler.similarity(s0, s1)

In [4]:
from strsimpy.jaccard import Jaccard

def getJaccard(s0, s1):
    jaccard = Jaccard(2)
    return 1-jaccard.distance(s0, s1)

In [5]:
def calcMatrix(a0, a1, f):
    return [[f(e0,e1) for e1 in a1] for e0 in a0]

import seaborn as sns

def getTable(city1, city2, method):
    m = calcMatrix(city1.headers, city2.headers, method.function)
    df = pd.DataFrame(data=m, columns=city2.headers, index=city1.headers)
    df.caption = city1.name + " - " + city2.name
    return df

def getStyledTable(df):
    caption = df.caption
    cm = sns.light_palette("green", as_cmap=True)
    df = df.clip(0, 1)
    return df.style.background_gradient(cmap=cm, high=1, low=0)
    


In [10]:
headers = [la_headers, london_headers, baltimore_headers, global_headers]
cities = [london, la, baltimore]

class Method():
    def __init__(self, name, function):
        self.name = name
        self.function = function

jaccard = Method("Jaccard", getJaccard)
jaroWinkler = Method("Jaro-Winkler", getJaroWinkler)

methods = [jaccard, jaroWinkler]


for method in methods:
    display(HTML("<h2>%s</h2>" % method.name))
    for city in cities:
        display(HTML("<h3>Global - %s</h3>" % city.name))
        display(getStyledTable(getTable(glob, city, method)))

,soa_code,borough,major_category,minor_category,value,year,month
id,0,0,0,0,0,0,0
type,0,0,0,0,0,0,0
date,0,0,0.153846,0.153846,0,0,0
city,0,0,0,0,0,0,0
district,0,0,0,0,0,0,0
weapon,0,0,0,0,0,0.142857,0.125


,DR Number,Date Reported,Date Occurred,Time Occurred,Area ID,Area Name,Reporting District,Crime Code,Crime Code Description,MO Codes,Victim Age,Victim Sex,Victim Descent,Premise Code,Premise Description,Weapon Used Code,Weapon Description,Status Code,Status Description,Crime Code 1,Crime Code 2,Crime Code 3,Crime Code 4,Address,Cross Street,Location
id,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
type,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
date,0,0.166667,0.153846,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0833333,0.0526316,0,0,0,0,0,0,0.111111
city,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
district,0,0,0,0,0,0,0.333333,0.0666667,0.04,0,0.142857,0.142857,0.111111,0.0588235,0.0869565,0,0.0454545,0,0.0434783,0.0625,0.0625,0.0625,0.0625,0,0.0588235,0
weapon,0,0.0666667,0,0,0.1,0.0833333,0.047619,0,0.0434783,0,0,0,0,0,0.0454545,0.25,0.235294,0,0.047619,0,0,0,0,0,0,0.0909091


,CrimeDate,CrimeTime,CrimeCode,Location,Description,Inside/Outside,Weapon,Post,District,Neighborhood,Longitude,Latitude,Location 1,Premise,vri_name1,Total Incidents
id,0,0,0,0,0,0.1,0,0,0,0,0,0,0,0,0,0.0714286
type,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
date,0.222222,0,0,0.111111,0,0,0,0,0,0,0,0.111111,0.0909091,0,0,0
city,0,0,0,0,0,0,0,0,0,0,0.1,0.111111,0,0,0,0.0625
district,0.0714286,0.0833333,0.0714286,0,0.0625,0,0,0.111111,0.75,0,0,0,0,0.0833333,0.0714286,0
weapon,0,0,0,0.0909091,0.0714286,0,0.666667,0,0,0,0.0833333,0,0.0769231,0,0,0


,soa_code,borough,major_category,minor_category,value,year,month
id,0,0,0,0.52381,0,0,0
type,0,0,0.440476,0.440476,0.483333,0.5,0
date,0.458333,0,0.654762,0.654762,0.633333,0.5,0.483333
city,0,0,0.547619,0.543651,0,0,0.483333
district,0.5,0.422619,0.418651,0.511905,0,0.458333,0.441667
weapon,0.361111,0.436508,0.492063,0.460317,0.455556,0.611111,0


,DR Number,Date Reported,Date Occurred,Time Occurred,Area ID,Area Name,Reporting District,Crime Code,Crime Code Description,MO Codes,Victim Age,Victim Sex,Victim Descent,Premise Code,Premise Description,Weapon Used Code,Weapon Description,Status Code,Status Description,Crime Code 1,Crime Code 2,Crime Code 3,Crime Code 4,Address,Cross Street,Location
id,0,0,0,0.525641,0,0,0.518519,0.533333,0.69697,0,0.533333,0.533333,0.52381,0.527778,0.517544,0,0,0,0,0.527778,0.527778,0.527778,0.527778,0.547619,0,0
type,0,0.549145,0.551282,0.442308,0.464286,0.453704,0.527778,0.45,0.431818,0.458333,0.45,0.45,0.547619,0.444444,0.434211,0.375,0.37037,0.44697,0.537037,0.444444,0.444444,0.444444,0.444444,0.464286,0,0
date,0,0.660256,0.660256,0.442308,0.428571,0.407407,0.37037,0.45,0.363636,0.458333,0.45,0.45,0.547619,0.444444,0.434211,0.375,0.37037,0.393939,0.527778,0.444444,0.444444,0.444444,0.444444,0.595238,0.444444,0.583333
city,0,0.442308,0.442308,0.442308,0,0,0.37037,0.45,0.431818,0,0.572222,0.572222,0.543651,0.444444,0.434211,0,0,0.44697,0.435185,0.444444,0.444444,0.444444,0.444444,0,0.444444,0.583333
district,0,0.535256,0.424145,0.424145,0.422619,0.412037,0.567593,0.316667,0.477273,0,0.55,0.55,0.594048,0.430556,0.577485,0.395833,0.567593,0.438131,0.583333,0.305556,0.305556,0.305556,0.305556,0.490079,0.527778,0.5
weapon,0,0.574786,0.32906,0.41453,0.539683,0.518519,0.62963,0.511111,0.474747,0.430556,0,0,0,0.5,0.406433,0.715278,0.703704,0.505051,0.314815,0.5,0.5,0.5,0.5,0,0.416667,0.513889


,CrimeDate,CrimeTime,CrimeCode,Location,Description,Inside/Outside,Weapon,Post,District,Neighborhood,Longitude,Latitude,Location 1,Premise,vri_name1,Total Incidents
id,0.537037,0.537037,0.537037,0,0,0.714286,0,0,0.541667,0.527778,0,0.541667,0,0,0.537037,0
type,0.453704,0.453704,0.453704,0,0.393939,0.440476,0.444444,0,0.458333,0.444444,0,0.458333,0.45,0.464286,0,0.438889
date,0.453704,0.453704,0.453704,0.583333,0.44697,0.547619,0.444444,0.5,0.458333,0.444444,0.453704,0.583333,0.566667,0.464286,0,0.377778
city,0.453704,0.453704,0.453704,0.583333,0.560606,0.440476,0,0.5,0.583333,0.444444,0.574074,0.416667,0.572222,0,0.453704,0.438889
district,0.458333,0.458333,0.324074,0.5,0.598485,0.594048,0,0.583333,0.916667,0.472222,0.490741,0.472222,0.447222,0.490079,0.324074,0.505556
weapon,0.425926,0.425926,0.518519,0.513889,0.505051,0.325397,0.888889,0,0,0.5,0.518519,0.430556,0.488889,0.436508,0.351852,0.455556


In [7]:
pd.DataFrame(data=headers, index=['la', 'london', 'baltimore', 'global'])

,0,1,2,3,4,5,6,7,8,9,...,16,17,18,19,20,21,22,23,24,25
la,DR Number,Date Reported,Date Occurred,Time Occurred,Area ID,Area Name,Reporting District,Crime Code,Crime Code Description,MO Codes,...,Weapon Description,Status Code,Status Description,Crime Code 1,Crime Code 2,Crime Code 3,Crime Code 4,Address,Cross Street,Location
london,soa_code,borough,major_category,minor_category,value,year,month,None,None,None,...,None,None,None,None,None,None,None,None,None,None
baltimore,CrimeDate,CrimeTime,CrimeCode,Location,Description,Inside/Outside,Weapon,Post,District,Neighborhood,...,None,None,None,None,None,None,None,None,None,None
global,id,type,date,city,district,weapon,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None


In [8]:
def mergeBy(df1, df2, f):
    return df1.combine(df2, f)
    
maximum = lambda s1, s2: s1 if s1 > s2 else s2
minimum = lambda s1, s2: s1 if s1 < s2 else s2
avgerage = lambda s1, s2: (s1 + s2) / 2

import numpy as np

In [9]:
for city in cities:
    t1 = getTable(glob, city, jaccard)
    t2 = getTable(glob, city, jaroWinkler)
    display(HTML("<h3>%s - maximum</h2>" % city.name))
    display(mergeBy(t1, t2, np.maximum))
    display(HTML("<h3>%s - minimum</h2>" % city.name))
    display(mergeBy(t1, t2, np.minimum))
    display(HTML("<h3>%s - average</h2>" % city.name))
    display(mergeBy(t1, t2, avgerage))

,soa_code,borough,major_category,minor_category,value,year,month
id,0.000000,0.000000,0.000000,0.523810,0.000000,0.000000,0.000000
type,0.000000,0.000000,0.440476,0.440476,0.483333,0.500000,0.000000
date,0.458333,0.000000,0.654762,0.654762,0.633333,0.500000,0.483333
city,0.000000,0.000000,0.547619,0.543651,0.000000,0.000000,0.483333
district,0.500000,0.422619,0.418651,0.511905,0.000000,0.458333,0.441667
weapon,0.361111,0.436508,0.492063,0.460317,0.455556,0.611111,0.125000


,soa_code,borough,major_category,minor_category,value,year,month
id,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0
type,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0
date,0.0,0.0,0.153846,0.153846,0.0,0.000000,0.0
city,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0
district,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0
weapon,0.0,0.0,0.000000,0.000000,0.0,0.142857,0.0


,soa_code,borough,major_category,minor_category,value,year,month
id,0.000000,0.000000,0.000000,0.261905,0.000000,0.000000,0.000000
type,0.000000,0.000000,0.220238,0.220238,0.241667,0.250000,0.000000
date,0.229167,0.000000,0.404304,0.404304,0.316667,0.250000,0.241667
city,0.000000,0.000000,0.273810,0.271825,0.000000,0.000000,0.241667
district,0.250000,0.211310,0.209325,0.255952,0.000000,0.229167,0.220833
weapon,0.180556,0.218254,0.246032,0.230159,0.227778,0.376984,0.062500


,DR Number,Date Reported,Date Occurred,Time Occurred,Area ID,Area Name,Reporting District,Crime Code,Crime Code Description,MO Codes,...,Weapon Description,Status Code,Status Description,Crime Code 1,Crime Code 2,Crime Code 3,Crime Code 4,Address,Cross Street,Location
id,0.0,0.000000,0.000000,0.525641,0.000000,0.000000,0.518519,0.533333,0.696970,0.000000,...,0.000000,0.000000,0.000000,0.527778,0.527778,0.527778,0.527778,0.547619,0.000000,0.000000
type,0.0,0.549145,0.551282,0.442308,0.464286,0.453704,0.527778,0.450000,0.431818,0.458333,...,0.370370,0.446970,0.537037,0.444444,0.444444,0.444444,0.444444,0.464286,0.000000,0.000000
date,0.0,0.660256,0.660256,0.442308,0.428571,0.407407,0.370370,0.450000,0.363636,0.458333,...,0.370370,0.393939,0.527778,0.444444,0.444444,0.444444,0.444444,0.595238,0.444444,0.583333
city,0.0,0.442308,0.442308,0.442308,0.000000,0.000000,0.370370,0.450000,0.431818,0.000000,...,0.000000,0.446970,0.435185,0.444444,0.444444,0.444444,0.444444,0.000000,0.444444,0.583333
district,0.0,0.535256,0.424145,0.424145,0.422619,0.412037,0.567593,0.316667,0.477273,0.000000,...,0.567593,0.438131,0.583333,0.305556,0.305556,0.305556,0.305556,0.490079,0.527778,0.500000
weapon,0.0,0.574786,0.329060,0.414530,0.539683,0.518519,0.629630,0.511111,0.474747,0.430556,...,0.703704,0.505051,0.314815,0.500000,0.500000,0.500000,0.500000,0.000000,0.416667,0.513889


,DR Number,Date Reported,Date Occurred,Time Occurred,Area ID,Area Name,Reporting District,Crime Code,Crime Code Description,MO Codes,...,Weapon Description,Status Code,Status Description,Crime Code 1,Crime Code 2,Crime Code 3,Crime Code 4,Address,Cross Street,Location
id,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,...,0.000000,0.000000,0.000000,0.0000,0.0000,0.0000,0.0000,0.0,0.000000,0.000000
type,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,...,0.000000,0.000000,0.000000,0.0000,0.0000,0.0000,0.0000,0.0,0.000000,0.000000
date,0.0,0.166667,0.153846,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,...,0.000000,0.083333,0.052632,0.0000,0.0000,0.0000,0.0000,0.0,0.000000,0.111111
city,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,...,0.000000,0.000000,0.000000,0.0000,0.0000,0.0000,0.0000,0.0,0.000000,0.000000
district,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.333333,0.066667,0.040000,0.0,...,0.045455,0.000000,0.043478,0.0625,0.0625,0.0625,0.0625,0.0,0.058824,0.000000
weapon,0.0,0.066667,0.000000,0.0,0.1,0.083333,0.047619,0.000000,0.043478,0.0,...,0.235294,0.000000,0.047619,0.0000,0.0000,0.0000,0.0000,0.0,0.000000,0.090909


,DR Number,Date Reported,Date Occurred,Time Occurred,Area ID,Area Name,Reporting District,Crime Code,Crime Code Description,MO Codes,...,Weapon Description,Status Code,Status Description,Crime Code 1,Crime Code 2,Crime Code 3,Crime Code 4,Address,Cross Street,Location
id,0.0,0.000000,0.000000,0.262821,0.000000,0.000000,0.259259,0.266667,0.348485,0.000000,...,0.000000,0.000000,0.000000,0.263889,0.263889,0.263889,0.263889,0.273810,0.000000,0.000000
type,0.0,0.274573,0.275641,0.221154,0.232143,0.226852,0.263889,0.225000,0.215909,0.229167,...,0.185185,0.223485,0.268519,0.222222,0.222222,0.222222,0.222222,0.232143,0.000000,0.000000
date,0.0,0.413462,0.407051,0.221154,0.214286,0.203704,0.185185,0.225000,0.181818,0.229167,...,0.185185,0.238636,0.290205,0.222222,0.222222,0.222222,0.222222,0.297619,0.222222,0.347222
city,0.0,0.221154,0.221154,0.221154,0.000000,0.000000,0.185185,0.225000,0.215909,0.000000,...,0.000000,0.223485,0.217593,0.222222,0.222222,0.222222,0.222222,0.000000,0.222222,0.291667
district,0.0,0.267628,0.212073,0.212073,0.211310,0.206019,0.450463,0.191667,0.258636,0.000000,...,0.306524,0.219066,0.313406,0.184028,0.184028,0.184028,0.184028,0.245040,0.293301,0.250000
weapon,0.0,0.320726,0.164530,0.207265,0.319841,0.300926,0.338624,0.255556,0.259113,0.215278,...,0.469499,0.252525,0.181217,0.250000,0.250000,0.250000,0.250000,0.000000,0.208333,0.302399


,CrimeDate,CrimeTime,CrimeCode,Location,Description,Inside/Outside,Weapon,Post,District,Neighborhood,Longitude,Latitude,Location 1,Premise,vri_name1,Total Incidents
id,0.537037,0.537037,0.537037,0.000000,0.000000,0.714286,0.000000,0.000000,0.541667,0.527778,0.000000,0.541667,0.000000,0.000000,0.537037,0.071429
type,0.453704,0.453704,0.453704,0.000000,0.393939,0.440476,0.444444,0.000000,0.458333,0.444444,0.000000,0.458333,0.450000,0.464286,0.000000,0.438889
date,0.453704,0.453704,0.453704,0.583333,0.446970,0.547619,0.444444,0.500000,0.458333,0.444444,0.453704,0.583333,0.566667,0.464286,0.000000,0.377778
city,0.453704,0.453704,0.453704,0.583333,0.560606,0.440476,0.000000,0.500000,0.583333,0.444444,0.574074,0.416667,0.572222,0.000000,0.453704,0.438889
district,0.458333,0.458333,0.324074,0.500000,0.598485,0.594048,0.000000,0.583333,0.916667,0.472222,0.490741,0.472222,0.447222,0.490079,0.324074,0.505556
weapon,0.425926,0.425926,0.518519,0.513889,0.505051,0.325397,0.888889,0.000000,0.000000,0.500000,0.518519,0.430556,0.488889,0.436508,0.351852,0.455556


,CrimeDate,CrimeTime,CrimeCode,Location,Description,Inside/Outside,Weapon,Post,District,Neighborhood,Longitude,Latitude,Location 1,Premise,vri_name1,Total Incidents
id,0.000000,0.000000,0.000000,0.000000,0.000000,0.1,0.000000,0.000000,0.00,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000
type,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000
date,0.222222,0.000000,0.000000,0.111111,0.000000,0.0,0.000000,0.000000,0.00,0.0,0.000000,0.111111,0.090909,0.000000,0.000000,0.0000
city,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.0,0.100000,0.111111,0.000000,0.000000,0.000000,0.0625
district,0.071429,0.083333,0.071429,0.000000,0.062500,0.0,0.000000,0.111111,0.75,0.0,0.000000,0.000000,0.000000,0.083333,0.071429,0.0000
weapon,0.000000,0.000000,0.000000,0.090909,0.071429,0.0,0.666667,0.000000,0.00,0.0,0.083333,0.000000,0.076923,0.000000,0.000000,0.0000


,CrimeDate,CrimeTime,CrimeCode,Location,Description,Inside/Outside,Weapon,Post,District,Neighborhood,Longitude,Latitude,Location 1,Premise,vri_name1,Total Incidents
id,0.268519,0.268519,0.268519,0.000000,0.000000,0.407143,0.000000,0.000000,0.270833,0.263889,0.000000,0.270833,0.000000,0.000000,0.268519,0.035714
type,0.226852,0.226852,0.226852,0.000000,0.196970,0.220238,0.222222,0.000000,0.229167,0.222222,0.000000,0.229167,0.225000,0.232143,0.000000,0.219444
date,0.337963,0.226852,0.226852,0.347222,0.223485,0.273810,0.222222,0.250000,0.229167,0.222222,0.226852,0.347222,0.328788,0.232143,0.000000,0.188889
city,0.226852,0.226852,0.226852,0.291667,0.280303,0.220238,0.000000,0.250000,0.291667,0.222222,0.337037,0.263889,0.286111,0.000000,0.226852,0.250694
district,0.264881,0.270833,0.197751,0.250000,0.330492,0.297024,0.000000,0.347222,0.833333,0.236111,0.245370,0.236111,0.223611,0.286706,0.197751,0.252778
weapon,0.212963,0.212963,0.259259,0.302399,0.288240,0.162698,0.777778,0.000000,0.000000,0.250000,0.300926,0.215278,0.282906,0.218254,0.175926,0.227778
